In [204]:
# %reset -f

'''
This python script modifies the elastic network in a supplied itp file
Input:
-i [required] Name of input itp file.
-o [optional] Name of output itp file.
-l [optional] Name of output log file containing removed elastic network lines. A log file will not be created unless a name has been given.
------------
At least one of the following commands must be used before the script will run.
For the following commands: All residus mentioned by the commands are included in the selection.
------------
-ri [optional] Remove internal elastic networks. Examples:
    -ri R:245:282" Removes all elastic networks between all residues from residue 245 to residue 282.
    -ri R:245:282-E:266:279 Same as above except residue 266 to 279 are exempt from the selection.

-re [optional] Remove external elastic networks. Examples:
    -re R:245:282 Removes all elastic networks between any residue from 245 to 282 and all other residues.
    -re R:245:282-E:266:279 Same as above except residue 266 to 279 are exempt from the selection.
    -re R:245:282-E:400:450 Same as first example except residue 400 to 450 are exempt from the non-selected residues.

-rb [optional] Remove elastic networks between two groups of residues. Examples:
    -rb R:30:244,R:283:500 Removes all elastic network between the two groups of residues.
    (group 1: 30 to 244 and group 2: 283 to 500)
    -rb R:30:244,R:283:500-E:350:400 Removes all elastic network between the two groups of residues.
    (group 1: 30 to 244 and group 2: 283 to 349 and 401 to 500)
'''

import os.path
import argparse
import sys
import numpy as np
# import scipy as spy
from scipy.spatial import distance

### Parser
# parser = argparse.ArgumentParser(formatter_class=argparse.RawTextHelpFormatter)
# parser = argparse.ArgumentParser()

# ### Optional arguments
# parser.add_argument("-o", dest = "output_name", default = None,
#                     help = "Name of output itp file.")

# parser.add_argument("-l", dest = "log_name", default = None,
#                     help = "Name of output log file containing removed elastic network lines.\n"
#                     "A log file will not be created unless a name has been given.\n")

# ### Remove networks flags
# # Remove internal networks
# parser.add_argument('-ri', dest = "remove_internal", action='append', default = [], nargs='+',
#                     help = "Removes internal elastic networks. Examples:\n"
#                     "-ri R:245:282 Removes all elastic networks between all residues from residue 245 to residue 282.\n"
#                     "-ri R:245:282-E:266:279 Same as above except residue 266 to 279 are exempt from the selection.\n")

# # Remove external networks
# parser.add_argument('-re', dest = "remove_external", action='append', default = [], nargs='+',
#                     help = "Removes external elastic networks. Examples:\n"
#                     "-re R:266:279 Removes all elastic networks between any residue from 266 to 279 and all other residues.\n"
#                     "-re R:266:279-E:270:275 Same as above except residue 270 to 275 are exempt from the selection.\n"
#                     "-re R:266:279-E:400:450 Same as first example except residue 400 to 450 are exempt from the non-selected residues.\n")

# # Remove networks between two groups
# parser.add_argument('-rb', dest = "remove_between", action='append', default = [], nargs='+',
#                     help = "Removes elastic networks between two groups of residues. Examples:\n"
#                     "-rb R:30:244,R:283:500 Removes all elastic network between the two groups of residues.\n"
#                     "(group 1: 30 to 244 and group 2: 283 to 500)\n"
#                     "-rb R:30:244-E40:50,R:283:500-E:350:400 Removes all elastic network between the two groups of residues.\n"
#                     "(group 1: 30 to 39 and 41 to 244 and group 2: 283 to 349 and 401 to 500)\n")

# # Remove all networks associated with this residue selection
# parser.add_argument('-ra', dest = "remove_all", action='append', default = [], nargs='+',
#                     help = "Removes all elastic networks associated with this selection.\n")

########################################## NOT IMPLEMENTED
### # ### Modify networks
### # # Modify Fc and/or distance
### # parser.add_argument('-ms', dest = "modify_single", action='append', default = [], nargs='+',
### #                     help = "Add elastic networks between two groups of residues. Examples:\n"
### #                     "-as M:280:433,dis:0.95,fc:700 Changes the Fc and distance networks between the two residues.\n"
### #                     "dis designates the distance for the bond. If not distance given, then it will be"
### #                     "calculated based on pdb file. fc designates the force constant of the bond\n")

### # Modify multiple
### # parser.add_argument('-mm', dest = "modify_multiple", action='append', default = [], nargs='+',
### #                     help = "Add elastic networks between two groups of residues. Examples:\n"
### #                     "-as M:280:433,dis:0.95,fc:700 Changes the Fc and distance networks between the two residues.\n"
### #                     "dis designates the distance for the bond. If not distance given, then it will be"
### #                     "calculated based on pdb file. fc designates the force constant of the bond\n")
##########################################

# ### Add networks
# # Input pdb file
# parser.add_argument('-f', dest = "struc_name", default = None,
#                     help = "A pdb or gro file is required to add networks based on pre-existing distances\n"
#                      "Protein must not stretch across the periodic boundary\n")

# # Add a single bond between two residues
# parser.add_argument('-as', dest = "add_single", action='append', default = [], nargs='+',
#                     help = "Add elastic networks between two groups of residues. Examples:\n"
#                     "-as A:280:433,dis:0.95,fc:700 Adds elastic networks between the two residues.\n"
#                     "dis designates the distance for the bond. If no distance is given, then it will be"
#                     "calculated based on pdb file. fc designates the force constant of the bond (default = 700)\n")

# # Generate elastic network for range of residues (between selection and whole itp)
# parser.add_argument('-aa', dest = "add_automatic", action='append', default = [], nargs='+',
#                     help = "Add elastic networks between two groups of residues. Examples:\n"
#                     "Requires pdb file. Distances will be based on said pdb file.\n"
#                     "-aa A:50:75,eu:0.85,el:0.5,fc:700,replace\n"
#                     "Generates elastic networks for all of the residues in the selection.\n"
#                     "eu, el and fc sets the upper limit, lower limit and force constant.\n"
#                     "replace designates that existing networks should be replaced instead of skipped.\n")

# ### Arguments required for script to run
# requiredNamed = parser.add_argument_group('required arguments')
# requiredNamed.add_argument('-i', dest = "input_name",
#                            help = "Name of input itp file.", required = True)

# ### Print parser help if no flags provided:
# if len(sys.argv) == 1:
#     parser.print_help()
#     sys.exit()

# ### Parses the arguments (checks if required arguments are present)
# args = parser.parse_args()

# ### Input file naming
# input_name = args.input_name

# ### Networks to be removed:
# rem_int_input = args.remove_internal
# rem_ext_input = args.remove_external
# rem_btw_input = args.remove_between
# rem_all_input = args.remove_all

# ### Networks to be added:
# add_sin_input = args.add_single
# add_aut_input = args.add_automatic

# ### Output file naming
# if args.output_name != None:
#     output_name = args.output_name
# else:
#     output_name = input_name[:-4] + "_modified" + ".itp"

# ### structure file (pdb or gro)
# if args.struc_name != None:
#     struc_name = args.struc_name

# ### Log file naming
# log_file = []
# log_file.extend("Output itp file: " + output_name + "\n")
# create_log = False
# if args.log_name != None:
#     create_log = True
#     log_name = args.log_name
#     log_file.extend("Log file: " + log_name + "\n")


### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### Parser Handling done




### ### ### ### ### ### ### ### ### ### Test files

input_name = "martini3_ElNet_modifier_test_files/AtSUC1_res152_standard.itp"
output_name = "martini3_ElNet_modifier_test_files/AtSUC1_res152_standard_modified.itp"
struc_name = "martini3_ElNet_modifier_test_files/04_output_martinize.pdb"

rem_int_input = [[]]
rem_ext_input = [[]]
rem_btw_input = [[]]
rem_all_input = [[]]

add_sin_input = [[]]
add_aut_input = [[]]

# rem_int_input = [["R:245:282-E:266:279"]]
# rem_ext_input = [["R:245:282"], ["R:463:471"]]
# rem_ext_input = [["R:266:279"]]
# rem_btw_input = [["R:30:244,R:283:500"]]
rem_all_input = [["R:266:279"]]

add_sin_input = [["A:280:433,dis:0.95,fc:700"]]
add_aut_input = [["A:50:75,eu:0.85,el:0.5,fc:700,replace"]]

log_file = []
create_log = True
log_name = "martini3_ElNet_modifier_test_files/test.log"
log_file.extend("Log file: " + log_name + "\n")

### ### ### ### ### ### ### ### ### ### Test files over

print("Following elastic network removals were requested:")
if rem_int_input != [[]]:
    for i in rem_int_input:
        print("Internal: " + str(i))

if rem_ext_input != [[]]:
    for i in rem_ext_input:
        print("External: " + str(i))

if rem_btw_input != [[]]:
    for i in rem_btw_input:
        print("Between: " + str(i))

if rem_all_input != [[]]:
    for i in rem_all_input:
        if rem_int_input == [[]]:
            rem_int_input = rem_all_input
        else:
            rem_int_input.append(i)
        if rem_ext_input == [[]]:
            rem_ext_input = rem_all_input
        else:
            rem_ext_input.append(i)
        print("All: " + str(i))

print("Following elastic network additions were requested:")
if add_sin_input != [[]]:
    for i in add_sin_input:
        print("Single: " + str(i))

if add_aut_input != [[]]:
    for i in add_aut_input:
        print("Automatic: " + str(i))

### Open files
def file_reader(file_name):
    file = open(file_name, "r")
    file_read = [i for i in file]
    file.close()
    return file_read
itp_file = file_reader(input_name)

### Finds the appropriate lines for ElNet modifications
itp_separators_dict = {}
atom_line_found = False
ElNet_line_found = False
print("Processing itp file")
for line_number, line in enumerate(itp_file):
    ### [:-1] is to remove the \n from all lines
    if "[ atoms ]" in line[:-1]:
        itp_separators_dict["atoms_start"] = line_number + 1
        atom_line_found = True
    if atom_line_found == True and line[:-1] == "":
        itp_separators_dict["atoms_end"] = line_number
        atom_line_found = False
    
    if "Rubber band" in line[:-1]:
        itp_separators_dict["ElNet_start"] = line_number + 1
        ElNet_line_found = True
    if ElNet_line_found == True and line[:-1] == "":
        itp_separators_dict["ElNet_end"] = line_number
        ElNet_line_found = False

itp_separators_sorted_list = sorted([(name, line_nr) for name, line_nr in itp_separators_dict.items()], key=lambda x: x[1])

### Creates dictionary of residues and atoms from itp file
residue_dict = {}
list_of_atom_numbers = []

for atom in itp_file[itp_separators_dict["atoms_start"]:itp_separators_dict["atoms_end"]]:
    atom_split = atom.split()
    if str(atom_split[2]) not in residue_dict.keys() and "BB" in atom_split:
        residue_dict[str(atom_split[2])] = [atom_split[0]]
        list_of_atom_numbers.append(atom_split[0])
    elif str(atom_split[2]) in residue_dict.keys() and "BB" in atom_split:
        residue_dict[str(atom_split[2])].append(atom_split[0])
        list_of_atom_numbers.append(atom_split[0])

### Creates list of existing ElNets
ElNets = itp_file[itp_separators_dict["ElNet_start"]:itp_separators_dict["ElNet_end"]]


### Creates list of coordinates for each atom if structure file is present and array of distances between atoms
if struc_name != None:
    struc_file = file_reader(struc_name)
    ### If pdb file
    if struc_name[-3:] == "pdb":
        struc_file_split = [(line.split()[5], line.split()[1], line.split()[6], line.split()[7], line.split()[8]) for line in struc_file
                           if line.split()[0] == "ATOM"
                           and line.split()[1] in residue_dict[line.split()[5]]] ### Bead (atom) in residue dict
    ### If gro file
    if struc_name[-3:] == "gro":
        struc_file_split = [(line.split()[2], ''.join(filter(str.isdigit, line.split()[0])), line.split()[3], line.split()[4], line.split()[5]) for line in struc_file
                           if len(line) == 6 ### Correct length
                           and any([line.split()[1][:2] == "BB", line.split()[1][:2] == "SC"]) ### Checks if bead
                           and ''.join(filter(str.isdigit, line.split()[0])) in residue_dict[''.join(filter(str.isdigit, line.split()[0]))]] ### Bead (atom) in residue dict
    
    coordinates_list = [(float(x) * 0.1, float(y) * 0.1, float(z) * 0.1) for resid, atom, x, y, z in struc_file_split]
    
    distances_array = distance.cdist(coordinates_list, coordinates_list, "euclidean")
    distances_dict = {}
    for i in range(len(distances_array)):
        for j in range(len(distances_array)):
            if abs(int(i) - int(j)) > 2:
                distances_dict[(str(list_of_atom_numbers[i]), str(list_of_atom_numbers[j]))] = str(round(distances_array[i, j], 5))

### ### ### ### Functions
### Removes exemptions from atom list
def exemptions_remover(ElNet_list, exemptions):
    ### Finds all the atoms that are exempt from the selection
    exemptions_list = []
    for exemption in exemptions:
        rem_type, start_res, end_res = exemption.split(":")
        atom_exempt_list = [atom for res in range(int(start_res), int(end_res) + 1) for atom in residue_dict[str(res)]]
        exemptions_list.extend(atom_exempt_list)

    ### Removes exempt atoms from selection
    for atom_ex in exemptions_list:
        if atom_ex in ElNet_list:
            ElNet_list.remove(atom_ex)
    
    return ElNet_list

### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### Removals
### ### List containing all networks to be removed
print("Creating elastic network removal list")
Main_remover_list = []

### ### Writing requested changes to log file
log_requested_removals = []

### ### ### Internal elastic network removal calculator:
if rem_int_input != [[]]:
    for internal in rem_int_input:
        exemptions_present = False
        input_split = internal[0].split("-")
        
        log_requested_removals.append("-ri " + internal[0] + "\n")

        remove_list = input_split[0]
        if len(input_split) > 1:
            exemptions = input_split[1:]
            exemptions_present = True

        rem_type, start_res, end_res = remove_list.split(":")

        ### Finds all the atoms in the removal selection 
        atom_list = [str(atom) for res in range(int(start_res), int(end_res) + 1) for atom in residue_dict[str(res)]]

        ### Finds all the atoms that are exempt from the selection
        if exemptions_present == True:
            atom_list = exemptions_remover(ElNet_list = atom_list, exemptions = exemptions)

        Main_remover_list.extend([(atom_1, atom_2) for atom_1 in atom_list for atom_2 in atom_list if atom_1 != atom_2])

### ### ### External elastic network removal calculator:
if rem_ext_input != [[]]:
    for external in rem_ext_input:
        exemptions_present = False
        input_split = external[0].split("-")
        
        log_requested_removals.append("-re " + external[0] + "\n")
        
        remove_list = input_split[0]
        if len(input_split) > 1:
            exemptions = input_split[1:]
            exemptions_present = True

        rem_type, start_res, end_res = remove_list.split(":")

        ### Finds all the atoms in the selection
        atom_list = [str(atom) for res in range(int(start_res), int(end_res) + 1) for atom in residue_dict[str(res)]]

        ### Finds all atoms not in selection
        atom_ext_list = [str(atom) for res in list(residue_dict.keys()) for atom in residue_dict[str(res)] if atom not in atom_list]
        
        ### Finds all the atoms that are exempt from the selection
        if exemptions_present == True:
            atom_list = exemptions_remover(ElNet_list = atom_list, exemptions = exemptions)
            atom_ext_list = exemptions_remover(ElNet_list = atom_ext_list, exemptions = exemptions)
                
        Main_remover_list.extend([(atom_1, atom_2) for atom_1 in atom_list for atom_2 in atom_ext_list if atom_1 != atom_2])
        Main_remover_list.extend([(atom_1, atom_2) for atom_1 in atom_ext_list for atom_2 in atom_list if atom_1 != atom_2])

### ### ### Between groups elastic network removal calculator:
if rem_btw_input != [[]]:
    for between in rem_btw_input:
        group_atom_lists = []
        
        log_requested_removals.append("-rb " + between[0] + "\n")
        
        groups = [[group] for group in between[0].split(",")]
        for group in groups:
            exemptions_present = False
            input_split = group[0].split("-")

            remove_list = input_split[0]
            if len(input_split) > 1:
                exemptions = input_split[1:]
                exemptions_present = True

            rem_type, start_res, end_res = remove_list.split(":")

            ### Finds all the atoms in the selection
            atom_list = [str(atom) for res in range(int(start_res), int(end_res) + 1) for atom in residue_dict[str(res)]]

            ### Finds all the atoms that are exempt from the selection and non-selection
            if exemptions_present == True:
                atom_list = exemptions_remover(ElNet_list = atom_list, exemptions = exemptions)

            group_atom_lists.append(atom_list)

        Main_remover_list.extend([(atom_1, atom_2) for atom_1 in group_atom_lists[0] for atom_2 in group_atom_lists[1] if atom_1 != atom_2])
        Main_remover_list.extend([(atom_1, atom_2) for atom_1 in group_atom_lists[1] for atom_2 in group_atom_lists[0] if atom_1 != atom_2])

### Checks if removals have been requested
if len(Main_remover_list) != 0:

    print("Removal list calculated")
    ### Removes duplicates from list
    len_MRL_before = len(Main_remover_list)
    Main_remover_list = list(dict.fromkeys(Main_remover_list))
    len_MRL_after = len(Main_remover_list)

    if len_MRL_before - len_MRL_after != 0:
        print("Removed " + str(len_MRL_before - len_MRL_after) + " duplicate entries in the removal list")
    print("Will look for " + str(len(Main_remover_list)) + " Potential elastic network bonds")

    elastic_remove_counter = 0
    log_removed_networks = []
    for rubber_band in reversed(ElNets):
        rbs = rubber_band.split()
        if (rbs[0], rbs[1]) in Main_remover_list:
            elastic_remove_counter += 1
            ElNets.remove(rubber_band)
            log_removed_networks.append(rubber_band)

    print(str(elastic_remove_counter) + " elastic network bonds were removed")


### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### Additions
log_requested_additions = []
log_notice_additions = []
# if add_aut_input != [[]]:







if add_sin_input != [[]]:
    for single in add_sin_input:
        
        
        single_dict = {}
        
        for command in single[0].split(","):
            print(command)
            single_dict[command.split(":")[0]] = command.split(":")[1:]
        
        atom1, atom2 = residue_dict[single_dict["A"][0]][0], residue_dict[single_dict["A"][1]][0]
        
        log_requested_additions.append("-as " + single[0] + " relating to atom " + str(atom1) + " and " + str(atom2) + "\n")
        print(log_requested_additions[-1])
        
        ### Distance
        if not "dis" in single_dict.keys():
            assert (atom1, atom2) in distances_dict, \
                "Atom pair not found in distance dictionary. Are you sure they are more than 2 residues apart?"
            single_dict["dis"] = distances_dict[(atom1, atom2)]
            log_notice_additions.append("No distance given for " + "-as " + single[0] + ". Setting it to " + single_dict["dis"] + "\n")
            print(log_notice_additions[-1])
        else:
            single_dict["dis"] = single_dict["dis"][0] ### Otherwise will be a list with one string
        
        ### Force constant
        if not "fc" in single_dict.keys():
            single_dict["fc"] = "700"
            log_notice_additions.append("No force constant given for " + "-as " + single[0] + ". Setting it to " + single_dict["fc"] + "\n")
            print(log_notice_additions[-1])
        else:
            single_dict["fc"] = single_dict["fc"][0] ### Otherwise will be a list with one string
        
        ### Checking if bond already exists 
        log_notice_additions.append("Checking for existing bond to be replaced for " + "-as " + single[0] + "\n")
        print(log_notice_additions[-1])
        replaced_questionmark = False
        for line, rubber_band in enumerate(ElNets):
            rbs = rubber_band.split()
            if (rbs[0], rbs[1]) == (atom1, atom2):
                ElNets[line] = atom1 + " " + atom2 + " " + "1" + " " + single_dict["dis"] + " " + single_dict["fc"] + ".0" + "\n"
                replaced_questionmark = True
                log_notice_additions.append("Existing bond found. Replacing it.\n")
                print(log_notice_additions[-1])
                break
        if replaced_questionmark == False:
            ElNets.append(atom1 + " " + atom2 + " " + "1" + " " + single_dict["dis"] + " " + single_dict["fc"] + ".0" + "\n")
            log_notice_additions.append("No existing bond found. Creating a new one.\n")
            print(log_notice_additions[-1])
        
        print(single_dict)


### ### ### Log writer
log_file.extend("The following lines show the requested removals\n")
log_file.extend(log_requested_removals)
log_file.extend("The following lines show the requested additions and their notes\n")
log_file.extend(log_requested_additions)
log_file.extend(log_notice_additions)
log_file.extend(str(elastic_remove_counter) + " " + "elastic networks were removed\n")
log_file.extend("The following lines show the removed elastic networks\n")
log_file.extend(log_removed_networks)

### ### ### Output file handling
### Inserting new elastic network system file
before = itp_file[:itp_separators_dict["ElNet_start"]]
after = itp_file[itp_separators_dict["ElNet_end"]:]
new_itp_file = before + ElNets + after

### Checks if output file already exists and backs it up
def backupper(output_file_name):
    output_file_split = output_file_name.split("/")
    output_path = ""
    output_name = output_file_split[-1]
    if len(output_file_split) > 1:
        for i in range(len(output_file_split) - 1):
            output_path += output_file_split[i] + "/"
    if os.path.exists(output_file_name):
        print("File " + output_file_name + " already exists. Backing it up")
        number = 1
        while True:
            if os.path.exists(output_path + "#" + output_name + "." + str(number) + "#"):
                number += 1
            else:
                os.rename(output_file_name, output_path + "#" + output_name + "." + str(number) + "#")
                break

print("\n")
### Output itp file
backupper(output_name)
print("Writing output file: " + output_name)
new_file = open(output_name, "w")
for line in new_itp_file:
    new_file.write(line)
new_file.close()

### Output log file
if create_log == True:
    backupper(log_name)
    print("Writing log file: " + output_name)
    new_file = open(log_name, "w")
    for line in log_file:
        new_file.write(line)
    new_file.close()
print("\n")

Following elastic network removals were requested:
All: ['R:266:279']
Following elastic network additions were requested:
Single: ['A:280:433,dis:0.95,fc:700']
Automatic: ['A:50:75,eu:0.85,el:0.5,fc:700,replace']
Processing itp file
Creating elastic network removal list
Removal list calculated
Will look for 13118 Potential elastic network bonds
60 elastic network bonds were removed
A:280:433
dis:0.95
fc:700
-as A:280:433,dis:0.95,fc:700 relating to atom 607 and 960

Checking for existing bond to be replaced for -as A:280:433,dis:0.95,fc:700

No existing bond found. Creating a new one.

{'A': ['280', '433'], 'dis': '0.95', 'fc': '700'}


File martini3_ElNet_modifier_test_files/AtSUC1_res152_standard_modified.itp already exists. Backing it up
Writing output file: martini3_ElNet_modifier_test_files/AtSUC1_res152_standard_modified.itp
File martini3_ElNet_modifier_test_files/test.log already exists. Backing it up
Writing log file: martini3_ElNet_modifier_test_files/AtSUC1_res152_standard_mo

In [197]:
print(single_dict)
print(atom1, atom2)
print(list_of_atom_numbers[280 - 25])
print(residue_dict["280"])
print(list_of_atom_numbers[int(residue_dict["280"][0])])
print(max(residue_dict))
# print(list_of_atom_numbers)

{'A': ['280', '433'], 'dis': '0.95', 'fc': '700'}
677 1008
607
['607']


IndexError: list index out of range